**1. Use Spark and Databricks to run ELT processes by creating fact tables** | 
The fact table Python scripts should contain appropriate keys from the dimensions. In addition, the fact table scripts should appropriately generate the correct facts based on the diagrams provided in the first step. 

**2. Use Spark and Databricks to run ELT processes by creating dimension tables** | 
The dimension Python scripts should match the schema diagram. Dimensions should generate appropriate keys and should not contain facts. 

**3. Produce Spark code in Databricks using Jupyter Notebooks and Python scripts** | 
The transform scripts should at minimum adhere to the following: should write to delta; should use overwrite mode; **save as a table in delta**.

## BUSINESS OUTCOMES

1. Analyze how much time spend per ride, based on
   
   * [ ] date and time factors such as day of week and time of day 
   * which station is starting and / or ending station
   * age of the rider at time of the ride
   * whether the rider is a (paying) member or casual rider
   
 2. Analyze how much money is spend: 

  * per month, quarter, year 
  * per member, based on the age of the rider at account start

In [0]:
silver = "silver.riders"
gold = "dimRider"
silverdf = spark.sql(f"SELECT * FROM {silver}")
#silverdf
silverdf.show(5)
df = silverdf
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold}")

+----+--------+---------+----------+--------------------+-------------+-----------+---------+
 id| first| last| birthday| address|account_start|account_end|is_member|
+----+--------+---------+----------+--------------------+-------------+-----------+---------+
1000| Diana| Clark|1989-02-13| 1200 Alyssa Squares| 2019-04-23| null| true|
1001|Jennifer| Smith|1976-08-10| 397 Diana Ferry| 2019-11-01| 2020-09-01| true|
1002| Karen| Smith|1998-08-10|644 Brittany Row ...| 2022-02-04| null| true|
1003| Bryan| Roberts|1999-03-29|996 Dickerson Tur...| 2019-08-26| null| false|
1004| Jesse|Middleton|1969-04-11|7009 Nathan Expre...| 2019-09-14| null| true|
+----+--------+---------+----------+--------------------+-------------+-----------+---------+
only showing top 5 rows

In [0]:
silver = "silver.stations"
gold = "dimStation"

#create = "CREATE TABLE %s (id VARCHAR(50), name VARCHAR(150), \
#                           latitude FLOAT, longitude FLOAT )" % (gold)

silverdf = spark.sql(f"SELECT * FROM {silver}")
silverdf.show(5)
df = silverdf
#df.write.mode("overwrite").save(f"/delta/{gold}")
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold}")


+------------+--------------------+---------+----------+
 id| name| latitude| longitude|
+------------+--------------------+---------+----------+
 525|Glenwood Ave & To...| 42.0127| -87.66606|
KA1503000012| Clark St & Lake St|41.885796| -87.6311|
 637|Wood St & Chicago...|41.895634|-87.672066|
 13216| State St & 33rd St|41.834732|-87.625824|
 18003|Fairbanks St & Su...| 41.89581|-87.620255|
+------------+--------------------+---------+----------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import explode, sequence, to_date
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F

trips = "silver.trips"
gold = "dimDate"
(beginDate, endDate) = spark.sql(f"SELECT min (to_date(start_at)) as beginDate, add_months(max (to_date(ended_at)),12) as endDate FROM {trips}").first()
endDate = endDate + relativedelta(months=24)

spark.sql(f"select explode(sequence(to_timestamp('{beginDate}'), (to_timestamp('{endDate}')) , interval 1 hour)) as ts") \
    .createOrReplaceTempView('dates')

create = """
create or replace table dimDate
-- USING delta
-- LOCATION '/delta/dimDate'
as select
  ts,
  hour(ts) AS hour,
  dayofweek(ts) as dayofweek,
  dayofmonth(ts) as dayofmonth,
  weekofyear(ts) as weekofyear,
  month(ts) as month,
  quarter(ts) as quarter,
  year(ts) AS year
from
  dates
"""

spark.sql(create)
spark.sql("optimize dimDate zorder by (ts)")
df = spark.sql("select * from dimDate")
df.show(5)
# HMMM how to go from spark.sql table to df to write ? 
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold}")


+-------------------+----+---------+----------+----------+-----+-------+----+
 ts|hour|dayofweek|dayofmonth|weekofyear|month|quarter|year|
+-------------------+----+---------+----------+----------+-----+-------+----+
2021-02-01 00:00:00| 0| 2| 1| 5| 2| 1|2021|
2021-02-01 01:00:00| 1| 2| 1| 5| 2| 1|2021|
2021-02-01 02:00:00| 2| 2| 1| 5| 2| 1|2021|
2021-02-01 03:00:00| 3| 2| 1| 5| 2| 1|2021|
2021-02-01 04:00:00| 4| 2| 1| 5| 2| 1|2021|
+-------------------+----+---------+----------+----------+-----+-------+----+
only showing top 5 rows

In [0]:
silver = "silver.payments"
gold = "factPayment"

#create = "CREATE TABLE %s (id INT, date DATE, amount FLOAT, rider INT)" % (gold)
#spark.sql(create)

silverdf = spark.sql(f"SELECT * FROM {silver}")
silverdf.show(5)
df = silverdf
#df.write.mode("overwrite").save(f"/delta/{gold}")
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold}")

+---+----------+------+--------+
 id| date|amount|rider_id|
+---+----------+------+--------+
 1|2019-05-01| 9.0| 1000|
 2|2019-06-01| 9.0| 1000|
 3|2019-07-01| 9.0| 1000|
 4|2019-08-01| 9.0| 1000|
 5|2019-09-01| 9.0| 1000|
+---+----------+------+--------+
only showing top 5 rows

In [0]:
import pyspark.sql.functions as F

silver = "silver.trips"
rider = "silver.riders"
gold = "factTrip"

joineddf = spark.sql(f"""SELECT t.id, t.start_at, t.ended_at, t.duration, t.start_station, t.dest_station, t.rideable_type, t.rider_id, 
                                CAST (datediff (year, r.birthday, t.start_at) AS INTEGER) as rider_age 
                         FROM {silver} as t
                         LEFT JOIN {rider} as r ON t.rider_id = r.id
                      """)
df = joineddf.withColumn("duration",(F.col("ended_at").cast("int") - F.col("start_at").cast("int")))
df.show(5, truncate=False)
df.write.format("delta").mode("overwrite").saveAsTable(f"{gold}")

+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
id |start_at |ended_at |duration|start_station|dest_station|rideable_type|rider_id|rider_age|
+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
89E7AA6C29227EFF|2021-02-12 16:14:56|2021-02-12 16:21:43|407 |525 |660 |classic_bike |71934 |37 |
89E7AA6C29227EFF|2021-02-12 16:14:56|2021-02-12 16:21:43|407 |525 |660 |classic_bike |71934 |37 |
0FEFDE2603568365|2021-02-14 17:52:38|2021-02-14 18:12:09|1171 |525 |16806 |classic_bike |47854 |38 |
0FEFDE2603568365|2021-02-14 17:52:38|2021-02-14 18:12:09|1171 |525 |16806 |classic_bike |47854 |38 |
E6159D746B2DBB91|2021-02-09 19:10:18|2021-02-09 19:19:10|532 |KA1503000012 |TA1305000029|electric_bike|70870 |33 |
+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
only showing top 5 rows